In [108]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import random
import re
import pandas as pd
from selenium import webdriver
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

In [109]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time


options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

# 대기 시간
driver.implicitly_wait(3)

# 네이버 날씨 검색 URL
search_url = "https://search.naver.com/search.naver?query=고척날씨"
driver.get(search_url)

# 페이지 로딩을 위해 잠시 대기
time.sleep(3)

# 날짜 변환 함수
def convert_date(date_str, year):
    try:
        # 문자열에서 '일.월.' 형식으로 되어있는 날짜를 추출
        month, day = date_str.strip('.').split('.')
        # 현재 연도를 사용하거나 연도를 지정
        formatted_date = f"{year}/{month.zfill(2)}/{day.zfill(2)}"
        return formatted_date
    except Exception as e:
        print(f"날짜 변환 중 오류 발생: {e}")
        return None
# 날씨 정보 추출
weather_list = []
try:
    # 날씨 정보가 포함된 'ul' 요소 가져오기
    weather_items = driver.find_element(By.XPATH, "//*[@id='main_pack']/section[1]/div/div[2]/div[1]/div/div[1]/div/div[2]/ul")
    
    # 모든 날짜별 날씨 정보 가져오기
    weather_elements = weather_items.find_elements(By.XPATH, ".//li")  # 모든 'li' 요소 찾기

    # 현재 연도 (혹은 원하는 연도)
    current_year = "2024"
    for weather_element in weather_elements: 
        try:
            # 날짜 정보
            date_str = weather_element.find_element(By.XPATH, ".//div/div[1]/span/span").text
            date = convert_date(date_str, current_year)
            # 최저 기온
            min_temp_text = weather_element.find_element(By.XPATH, ".//div/div[3]/span/span[1]").text
            min_temperature = min_temp_text.replace('최저기온', '').strip().replace('\n', ' ')
            
            # 최고 기온
            max_temp_text = weather_element.find_element(By.XPATH, ".//div/div[3]/span/span[3]").text
            max_temperature = max_temp_text.replace('최고기온', '').strip().replace('\n', ' ')
            
            # 강수량
            rain = weather_element.find_element(By.XPATH, ".//div/div[2]/span[2]/span/span").text
            
            weather_list.append({
                "날짜": date,
                "위치": "고척",
                "최저기온": min_temperature,
                "최고기온": max_temperature,
                "강우확률": rain
            })
        except Exception as e:
            print(f"날씨 정보 추출 중 오류 발생: {e}")

finally:
    # 웹 드라이버 종료
    driver.quit()

weatherdf = pd.DataFrame(weather_list)
weatherdf

,날짜,위치,최저기온,최고기온,강우확률
0,2024/09/05,고척,24°,29°,60%
1,2024/09/06,고척,24°,31°,60%
2,2024/09/07,고척,23°,31°,20%
3,2024/09/08,고척,23°,31°,20%
4,2024/09/09,고척,23°,32°,20%
5,2024/09/10,고척,24°,32°,20%
6,2024/09/11,고척,24°,32°,30%
7,2024/09/12,고척,23°,31°,40%
8,2024/09/13,고척,22°,29°,40%
9,2024/09/14,고척,21°,28°,40%


In [110]:
import cx_Oracle
# 라이브러리 임포트
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Text, Date
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2

In [111]:
# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

# Oracle 데이터베이스 연결 문자열 생성
dsn_tns = cx_Oracle.makedsn(db_host, db_port, service_name=service_name)

# 데이터베이스 연결
connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=dsn_tns)

# 커서 생성
cursor = connection.cursor()

In [112]:
# 테이블 생성 쿼리
create_table_query = '''
create table WEEKWEATHER(
   날짜 Date,
   위치 VARCHAR2(50),
   최저기온 VARCHAR2(30),
   최고기온 VARCHAR2(30),
   강우확률 VARCHAR2(30)
)
'''
# 쿼리 실행
cursor.execute(create_table_query)

#변경 사항 커밋
connection.commit()

#커서와 연결 종료
cursor.close()
connection.close()

#성공 알람
print("Table created successfully!!")

In [ ]:
weatherdf.to_sql(
    'WEEKWEATHER', 
    con=engine, 
    if_exists='append',  # 기존 테이블에 데이터 추가
    index=False, 
    dtype={
        '날짜' :Date,
        '위치' :VARCHAR2(50),
        '최저기온': VARCHAR2(30),
        '최고기온': VARCHAR2(30),
        '강우확률': VARCHAR2(30)
    }
)
print("Table insert successfully")